In [1]:
import fitz
import time
import re
import os
import sys
from PIL import Image
from tqdm import tqdm_notebook as tqdm

print(fitz.__doc__)

PyMuPDF 1.13.20: Python bindings for the MuPDF 1.13.0 library,
built on 2018-09-09 09:54:14


In [2]:
def recoverpix(doc, item):
    x = item[0]  # xref of PDF image
    s = item[1]  # xref of its /SMask
    pix1 = fitz.Pixmap(doc, x)
    if s == 0:                    # has no /SMask
        return pix1               # no special handling
    pix2 = fitz.Pixmap(doc, s)    # create pixmap of /SMask entry
    # check that we are safe
    if not (pix1.irect == pix2.irect and \
            pix1.alpha == pix2.alpha == 0 and \
            pix2.n == 1):
        print("pix1", pix1, "pix2", pix2)
        #raise ValueError("unexpected situation")
    pix = fitz.Pixmap(pix1)       # copy of pix1, alpha channel added
    pix.setAlpha(pix2.samples)    # treat pix2.samples as alpha value
    pix1 = pix2 = None            # free temp pixmaps
    return pix

In [3]:
def pdf2pic(prefix, pdf_path, outputfolder_path):
    xreflist = [] 
    imgcount = 0 
    pdf = fitz.open(pdf_path)
    for i in range(len(pdf)):
        imglist = pdf.getPageImageList(i)
        for img in imglist:
            if img[0] in xreflist:             # this image has been processed
                continue 
            xreflist.append(img[0])            # take note of the xref
            if img[2]!=0 and img[3]!= 0:       #make sure the width or height of image is not zero.
                pix = recoverpix(pdf, img[:2]) # make pixmap from image
                if pix.n - pix.alpha < 4:      # can be saved as PNG
                    pass
                else:                          # must convert CMYK first
                    pix0 = fitz.Pixmap(fitz.csRGB, pix)
                    pix = pix0

                pic_name = prefix+"_p"+str(i)+img[7]+".png"
                pix.writePNG(pic_name)
                current_dir = os.path.join(os.getcwd(), pic_name)
                destination_dir = os.path.join(outputfolder_path, pic_name)
                if os.path.exists(destination_dir):
                    destination_dir += 'x'
                os.rename(current_dir, destination_dir)
                imgcount += 1
                pix = None    
    pdf.close()

In [4]:
def pdfs2pics(pdffolder_path, outputfolder_path):
    pdfs= os.listdir(pdffolder_path)
    for pdf in tqdm(pdfs): 
        if not os.path.isdir(pdf): #only open when it's not a folder
            prefix = pdf[0:40]
            pdf_path = os.path.join(pdffolder_path, pdf)
            pdf2pic(prefix, pdf_path, outputfolder_path)

In [5]:
pdf_folder = "PDFS"
current_path = os.getcwd()
pdffolder_path = os.path.join(current_path, pdf_folder)
outputfolder_path = os.path.join(current_path, "Pdf2Pic_output")
if os.path.exists(outputfolder_path):
    print("foler already existed, please create a new folder！")
    raise SystemExit
else:
    os.makedirs(outputfolder_path)
pdfs2pics(pdffolder_path, outputfolder_path)

1.pdf
img:  [288, 0, 169, 185, 8, 'DeviceRGB', '', 'Im1', 'DCTDecode']
img:  [289, 0, 158, 199, 8, 'DeviceRGB', '', 'Im2', 'DCTDecode']
img:  [14, 0, 952, 1105, 8, 'DeviceGray', '', 'Im3', 'DCTDecode']
img:  [29, 0, 1094, 938, 8, 'DeviceGray', '', 'Im4', 'DCTDecode']
img:  [56, 0, 950, 130, 8, 'DeviceGray', '', 'Im5', 'DCTDecode']
10.pdf
img:  [246, 0, 248, 272, 8, 'DeviceRGB', '', 'Im1', 'DCTDecode']
img:  [247, 0, 220, 295, 8, 'DeviceRGB', '', 'Im2', 'DCTDecode']
img:  [248, 0, 102, 102, 8, 'DeviceCMYK', '', 'Im3', 'DCTDecode']
img:  [28, 0, 900, 894, 8, 'DeviceRGB', '', 'Im4', 'DCTDecode']
img:  [40, 0, 1500, 775, 8, 'DeviceRGB', '', 'Im5', 'DCTDecode']
100.pdf
101.pdf
img:  [141, 0, 350, 87, 8, 'DeviceRGB', '', 'Im0', 'FlateDecode']
102.pdf
img:  [9, 0, 2508, 598, 8, 'DeviceCMYK', '', 'Im0', 'DCTDecode']
img:  [10, 0, 1426, 1526, 8, 'DeviceCMYK', '', 'Im1', 'DCTDecode']
img:  [155, 0, 350, 87, 8, 'DeviceRGB', '', 'Im0', 'FlateDecode']
103.pdf
img:  [1410, 0, 83, 83, 8, 'ICCBased', 

img:  [108, 0, 1429, 1178, 8, 'ICCBased', '', 'X1', 'FlateDecode']
img:  [118, 0, 1429, 1745, 8, 'ICCBased', '', 'X1', 'FlateDecode']
img:  [126, 0, 1429, 584, 8, 'ICCBased', '', 'X1', 'FlateDecode']
107.pdf
img:  [105, 0, 113, 101, 8, 'Separation', 'DeviceCMYK', 'Im1', 'DCTDecode']
img:  [106, 0, 338, 99, 8, 'DeviceGray', '', 'Im2', 'DCTDecode']
img:  [45, 0, 289, 63, 8, 'DeviceGray', '', 'Im3', 'DCTDecode']
img:  [7, 0, 1634, 2436, 8, 'DeviceCMYK', '', 'Im4', 'DCTDecode']
img:  [19, 0, 1999, 1999, 8, 'Separation', 'DeviceCMYK', 'Im5', 'DCTDecode']
108.pdf
img:  [6, 0, 1260, 1898, 8, 'DeviceRGB', '', 'Image_D158B1C3B71741589256CC61AC762038', 'DCTDecode']
img:  [60, 0, 355, 262, 8, 'Separation', 'ICCBased', 'Im1', 'DCTDecode']
img:  [346, 0, 438, 465, 8, 'Separation', 'ICCBased', 'Im2', 'DCTDecode']
img:  [400, 0, 638, 550, 8, 'DeviceGray', '', 'Im3', 'DCTDecode']
img:  [414, 0, 591, 840, 8, 'DeviceGray', '', 'Im4', 'DCTDecode']
img:  [446, 0, 591, 451, 8, 'DeviceGray', '', 'Im5', 'DCT

img:  [9874, 0, 22, 13, 8, 'Separation', 'ICCBased', 'Im119', 'FlateDecode']
img:  [9875, 0, 6, 14, 8, 'Separation', 'ICCBased', 'Im120', 'FlateDecode']
img:  [9876, 0, 14, 14, 8, 'Separation', 'ICCBased', 'Im121', 'FlateDecode']
img:  [9877, 0, 14, 14, 8, 'Separation', 'ICCBased', 'Im122', 'FlateDecode']
img:  [9878, 0, 6, 13, 8, 'Separation', 'ICCBased', 'X1', 'DCTDecode']
img:  [9879, 0, 33, 13, 8, 'Separation', 'ICCBased', 'Im100', 'FlateDecode']
img:  [9880, 0, 12, 14, 8, 'Separation', 'ICCBased', 'Im101', 'FlateDecode']
img:  [9881, 0, 49, 13, 8, 'Separation', 'ICCBased', 'Im102', 'FlateDecode']
img:  [9882, 0, 28, 13, 8, 'Separation', 'ICCBased', 'Im103', 'FlateDecode']
img:  [9883, 0, 22, 13, 8, 'Separation', 'ICCBased', 'Im104', 'FlateDecode']
img:  [9884, 0, 28, 14, 8, 'Separation', 'ICCBased', 'Im105', 'FlateDecode']
img:  [9885, 0, 39, 13, 8, 'Separation', 'ICCBased', 'Im106', 'FlateDecode']
img:  [9886, 0, 39, 13, 8, 'Separation', 'ICCBased', 'Im107', 'FlateDecode']
img:  

img:  [1052, 0, 5400, 7200, 1, 'DeviceGray', '', 'img10', 'JBIG2Decode']
img:  [1054, 0, 5400, 7200, 1, 'DeviceGray', '', 'img11', 'JBIG2Decode']
img:  [1056, 0, 5400, 7200, 1, 'DeviceGray', '', 'img12', 'JBIG2Decode']
img:  [1078, 0, 5400, 7200, 1, 'DeviceGray', '', 'img13', 'JBIG2Decode']
img:  [1080, 0, 5400, 7200, 1, 'DeviceGray', '', 'img14', 'JBIG2Decode']
126.pdf
img:  [69, 0, 93, 61, 8, 'Indexed', '', 'Im0', 'FlateDecode']
img:  [70, 0, 10, 50, 8, 'Indexed', '', 'Im1', 'FlateDecode']
img:  [71, 0, 93, 61, 8, 'Indexed', '', 'Im10', 'FlateDecode']
img:  [72, 0, 93, 61, 8, 'Indexed', '', 'Im11', 'FlateDecode']
img:  [73, 0, 93, 61, 8, 'Indexed', '', 'Im12', 'FlateDecode']
img:  [74, 0, 93, 61, 8, 'Indexed', '', 'Im13', 'FlateDecode']
img:  [75, 0, 10, 40, 8, 'Indexed', '', 'Im14', 'FlateDecode']
img:  [76, 0, 9, 50, 8, 'Indexed', '', 'Im15', 'FlateDecode']
img:  [77, 0, 93, 61, 8, 'Indexed', '', 'Im16', 'FlateDecode']
img:  [78, 0, 93, 61, 8, 'Indexed', '', 'Im17', 'FlateDecode']


img:  [31, 0, 493, 2, 8, 'ICCBased', '', 'X100', 'FlateDecode']
img:  [32, 0, 493, 2, 8, 'ICCBased', '', 'X101', 'FlateDecode']
img:  [33, 0, 493, 1, 8, 'ICCBased', '', 'X102', 'FlateDecode']
img:  [34, 0, 493, 2, 8, 'ICCBased', '', 'X103', 'FlateDecode']
img:  [35, 0, 493, 2, 8, 'ICCBased', '', 'X104', 'FlateDecode']
img:  [36, 0, 493, 1, 8, 'ICCBased', '', 'X105', 'FlateDecode']
img:  [37, 0, 493, 2, 8, 'ICCBased', '', 'X106', 'FlateDecode']
img:  [38, 0, 493, 1, 8, 'ICCBased', '', 'X107', 'FlateDecode']
img:  [39, 0, 493, 2, 8, 'ICCBased', '', 'X108', 'FlateDecode']
img:  [40, 0, 493, 2, 8, 'ICCBased', '', 'X109', 'FlateDecode']
img:  [41, 0, 493, 2, 8, 'ICCBased', '', 'X11', 'FlateDecode']
img:  [42, 0, 493, 1, 8, 'ICCBased', '', 'X110', 'FlateDecode']
img:  [43, 0, 493, 2, 8, 'ICCBased', '', 'X111', 'FlateDecode']
img:  [44, 0, 493, 2, 8, 'ICCBased', '', 'X112', 'FlateDecode']
img:  [45, 0, 493, 1, 8, 'ICCBased', '', 'X113', 'FlateDecode']
img:  [46, 0, 493, 2, 8, 'ICCBased', '', 

...

img:  [105, 0, 12, 16, 8, 'DeviceRGB', '', 'Im44', 'FlateDecode']
img:  [106, 0, 11, 16, 8, 'DeviceRGB', '', 'Im45', 'FlateDecode']
img:  [107, 0, 15, 16, 8, 'DeviceRGB', '', 'Im46', 'FlateDecode']
img:  [108, 0, 14, 16, 8, 'DeviceRGB', '', 'Im47', 'FlateDecode']
img:  [109, 0, 501, 48, 8, 'Indexed', '', 'Im48', 'FlateDecode']
img:  [110, 0, 11, 15, 8, 'DeviceRGB', '', 'Im10', 'FlateDecode']
img:  [111, 0, 12, 15, 8, 'DeviceRGB', '', 'Im11', 'FlateDecode']
img:  [112, 0, 10, 15, 8, 'DeviceRGB', '', 'Im12', 'FlateDecode']
img:  [113, 0, 14, 15, 8, 'DeviceRGB', '', 'Im13', 'FlateDecode']
img:  [114, 0, 13, 15, 8, 'DeviceRGB', '', 'Im14', 'FlateDecode']
img:  [115, 0, 2, 15, 8, 'DeviceRGB', '', 'Im15', 'FlateDecode']
img:  [116, 0, 501, 44, 8, 'Indexed', '', 'Im16', 'FlateDecode']
img:  [117, 0, 501, 36, 8, 'Indexed', '', 'Im17', 'FlateDecode']
img:  [118, 0, 12, 15, 8, 'DeviceRGB', '', 'Im18', 'FlateDecode']
img:  [119, 0, 14, 16, 8, 'DeviceRGB', '', 'Im19', 'FlateDecode']
img:  [120, 0,

img:  [794, 0, 107, 107, 8, 'DeviceCMYK', '', 'Im3', 'DCTDecode']
img:  [27, 0, 1800, 974, 8, 'DeviceRGB', '', 'Im4', 'DCTDecode']
img:  [53, 0, 1601, 1233, 8, 'DeviceRGB', '', 'Im5', 'DCTDecode']
img:  [92, 0, 900, 1344, 8, 'DeviceRGB', '', 'Im6', 'DCTDecode']
img:  [104, 0, 1301, 1287, 8, 'DeviceRGB', '', 'Im7', 'DCTDecode']
img:  [131, 0, 1350, 1337, 8, 'DeviceRGB', '', 'Im8', 'DCTDecode']
img:  [181, 0, 1601, 1214, 8, 'DeviceRGB', '', 'Im9', 'DCTDecode']
60.pdf
img:  [539, 0, 166, 182, 8, 'DeviceRGB', '', 'Im1', 'DCTDecode']
img:  [540, 0, 158, 199, 8, 'DeviceRGB', '', 'Im2', 'DCTDecode']
img:  [36, 0, 1334, 1308, 8, 'DeviceGray', '', 'Im3', 'DCTDecode']
img:  [57, 0, 697, 382, 8, 'DeviceRGB', '', 'Im4', 'DCTDecode']
img:  [58, 0, 684, 680, 8, 'DeviceGray', '', 'Im5', 'DCTDecode']
img:  [83, 0, 691, 636, 8, 'DeviceGray', '', 'Im6', 'DCTDecode']
img:  [170, 0, 671, 562, 8, 'DeviceGray', '', 'Im7', 'DCTDecode']
img:  [171, 0, 1334, 914, 8, 'DeviceGray', '', 'Im8', 'DCTDecode']
img:  

img:  [357, 0, 37, 36, 8, 'Indexed', '', 'Im61', 'FlateDecode']
img:  [358, 0, 71, 5, 8, 'Indexed', '', 'Im62', 'FlateDecode']
img:  [359, 0, 35, 35, 8, 'Indexed', '', 'Im63', 'FlateDecode']
img:  [360, 0, 34, 35, 8, 'Indexed', '', 'Im65', 'FlateDecode']
75.pdf
img:  [278, 0, 226, 151, 8, 'DeviceGray', '', 'Im1', 'DCTDecode']
img:  [20, 0, 1802, 1311, 1, '', '', 'Im2', 'CCITTFaxDecode']
img:  [25, 0, 1971, 1376, 1, '', '', 'Im3', 'CCITTFaxDecode']
img:  [37, 0, 620, 907, 8, 'DeviceGray', '', 'Im4', 'DCTDecode']
img:  [42, 0, 610, 849, 8, 'DeviceGray', '', 'Im5', 'DCTDecode']
img:  [59, 0, 948, 1179, 8, 'DeviceGray', '', 'Im6', 'DCTDecode']
img:  [67, 0, 603, 246, 8, 'DeviceGray', '', 'Im7', 'DCTDecode']
img:  [72, 0, 579, 876, 8, 'DeviceGray', '', 'Im8', 'DCTDecode']
img:  [77, 0, 632, 930, 8, 'DeviceGray', '', 'Im9', 'DCTDecode']
76.pdf
img:  [62, 0, 697, 654, 8, 'ICCBased', '', 'Im1', 'DCTDecode']
img:  [77, 0, 813, 377, 8, 'ICCBased', '', 'Im2', 'DCTDecode']
img:  [115, 0, 355, 1144

img:  [805, 1152, 32, 32, 8, 'ICCBased', '', 'Im175', 'FlateDecode']
img:  [671, 884, 32, 32, 8, 'ICCBased', '', 'Im176', 'FlateDecode']
img:  [732, 1006, 32, 32, 8, 'ICCBased', '', 'Im18', 'FlateDecode']
img:  [646, 834, 32, 32, 8, 'ICCBased', '', 'Im19', 'FlateDecode']
img:  [644, 830, 23, 23, 8, 'ICCBased', '', 'Im2', 'FlateDecode']
img:  [782, 1106, 32, 32, 8, 'ICCBased', '', 'Im20', 'FlateDecode']
img:  [679, 900, 68, 54, 8, 'ICCBased', '', 'Im21', 'FlateDecode']
img:  [649, 840, 32, 32, 8, 'ICCBased', '', 'Im22', 'FlateDecode']
img:  [731, 1004, 32, 32, 8, 'ICCBased', '', 'Im23', 'FlateDecode']
img:  [791, 1124, 32, 32, 8, 'ICCBased', '', 'Im24', 'FlateDecode']
img:  [754, 1050, 68, 54, 8, 'ICCBased', '', 'Im25', 'FlateDecode']
img:  [793, 1128, 32, 32, 8, 'ICCBased', '', 'Im26', 'FlateDecode']
img:  [750, 1042, 32, 32, 8, 'ICCBased', '', 'Im27', 'FlateDecode']
img:  [773, 1088, 32, 32, 8, 'ICCBased', '', 'Im28', 'FlateDecode']
img:  [730, 1002, 68, 54, 8, 'ICCBased', '', 'Im29',

img:  [161, 0, 865, 5, 8, 'Indexed', '', 'Im39', 'FlateDecode']
img:  [162, 0, 865, 1088, 8, 'Indexed', '', 'Im40', 'FlateDecode']
img:  [163, 0, 865, 2, 8, 'Indexed', '', 'Im41', 'FlateDecode']
img:  [164, 0, 865, 3, 8, 'Indexed', '', 'Im42', 'FlateDecode']
img:  [165, 0, 758, 1007, 8, 'Separation', 'ICCBased', 'Im43', 'DCTDecode']
img:  [166, 0, 288, 356, 8, 'Separation', 'ICCBased', 'Im6', 'FlateDecode']
88.pdf
img:  [14, 0, 1608, 1401, 8, 'DeviceRGB', '', 'Im0', 'DCTDecode']
img:  [144, 0, 246, 155, 8, 'DeviceRGB', '', 'Im0', 'FlateDecode']
89.pdf
img:  [4, 0, 758, 856, 8, 'Separation', 'DeviceCMYK', 'Im1', 'DCTDecode']
img:  [16, 0, 758, 627, 8, 'Separation', 'DeviceCMYK', 'Im2', 'DCTDecode']
img:  [34, 0, 465, 572, 8, 'DeviceCMYK', '', 'Im3', 'DCTDecode']
img:  [46, 0, 728, 457, 8, 'DeviceCMYK', '', 'Im4', 'DCTDecode']
img:  [60, 0, 465, 806, 8, 'DeviceCMYK', '', 'Im5', 'DCTDecode']
img:  [65, 0, 465, 404, 8, 'Separation', 'DeviceCMYK', 'Im6', 'DCTDecode']
img:  [77, 0, 465, 638,